<h1>7장 고급 텍스트 생성 기술과 도구</h1>
<i>프롬프트 엔지니어링을 넘어서</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter07.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 7장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
import os
import tqdm
from transformers.utils import logging

# tqdm 비활성화
tqdm.tqdm = lambda *args, **kwargs: iter([])
tqdm.auto.tqdm = lambda *args, **kwargs: iter([])
tqdm.notebook.tqdm = lambda *args, **kwargs: iter([])
os.environ["DISABLE_TQDM"] = "1"

logging.disable_progress_bar()

In [ ]:
%%capture
!pip install langchain_community langchain_openai duckduckgo-search

# 사용하는 파이썬과 CUDA 버전에 맞는 llama-cpp-python 패키지를 설치하세요.
# 현재 코랩의 파이썬 버전은 3.11이며 CUDA 버전은 12.4입니다.
!pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu124/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

# LLM 로드하기

In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

In [ ]:
from langchain import LlamaCpp

# 여러분의 컴퓨터에 다운로드한 모델의 경로를 입력하세요!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=4096,
    seed=42,
    verbose=False
)

In [ ]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

## 체인

In [ ]:
from langchain import PromptTemplate

# "input_prompt" 변수를 가진 프롬프트 템플릿을 만듭니다.
template = """<|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [ ]:
basic_chain = prompt | llm

In [ ]:
# 체인을 사용합니다.
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
    }
)

### 여러 템플릿을 가진 체인

In [ ]:
from langchain import LLMChain

# 이야기 제목을 위한 체인을 만듭니다.
template = """<|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [ ]:
title.invoke({"summary": "a girl that lost her mother"})

In [ ]:
# 요약과 제목을 사용하여 캐릭터 설명을 생성하는 체인을 만듭니다.
template = """<|user|>
Describe the main character of a story about {summary} with the title {title}.
Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [ ]:
# 요약, 제목, 캐릭터 설명을 사용해 이야기를 생성하는 체인을 만듭니다.
template = """<|user|>
Create a story about {summary} with the title {title}.
The main charachter is: {character}.
Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [ ]:
# 세 개의 요소를 연결하여 최종 체인을 만듭니다.
llm_chain = title | character | story

In [ ]:
llm_chain.invoke("a girl that lost her mother")

# 메모리

In [ ]:
# LLM에게 이름을 알려 줍니다.
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

In [ ]:
# LLM에게 이름을 묻습니다.
basic_chain.invoke({"input_prompt": "What is my name?"})

### 대화 버퍼

In [ ]:
# 대화 기록을 담을 수 있도록 프롬프트를 업데이트합니다.
template = """<|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [ ]:
from langchain.memory import ConversationBufferMemory

# 사용할 메모리를 정의합니다.
memory = ConversationBufferMemory(memory_key="chat_history")

# LLM, 프롬프트, 메모리를 연결합니다.
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
# 간단한 질문을 하여 대화 기록을 만듭니다.
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

In [ ]:
# LLM이 이름을 기억할까요?
llm_chain.invoke({"input_prompt": "What is my name?"})

### 윈도 대화 버퍼

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# 메모리에 마지막 두 개의 대화만 유지합니다.
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# LLM, 프롬프트, 메모리를 연결합니다.
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
# 두 개의 질문을 던져 메모리에 대화 기록을 저장합니다.
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

In [ ]:
# 이름을 기억하는고 있는지 확인합니다.
llm_chain.invoke({"input_prompt":"What is my name?"})

In [ ]:
# 이름을 기억하는고 있는지 확인합니다.
llm_chain.invoke({"input_prompt":"What is my age?"})

### 대화 요약

In [ ]:
# 요약 프롬프트 템플릿을 만듭니다.
summary_prompt_template = """<|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [ ]:
from langchain.memory import ConversationSummaryMemory

# 사용할 메모리를 정의합니다.
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# LLM, 프롬프트, 메모리를 연결합니다.
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
# 이름에 대해 질문하는 대화를 생성합니다.
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

In [ ]:
# 지금까지 내용이 요약되어 있는지 확인합니다.
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

In [ ]:
# 지금까지 요약 내용을 확인합니다.
memory.load_memory_variables({})

# 에이전트

In [ ]:
import os
from langchain_openai import ChatOpenAI

# 랭체인으로 오픈AI의 LLM을 로드합니다.
os.environ["OPENAI_API_KEY"] = "MY_KEY"
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# ReAct 템플릿을 만듭니다.
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# 에이전트에 전달할 도구를 만듭니다.
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# 도구를 준비합니다.
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# ReAct 에이전트를 만듭니다.
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
# 맥북 프로의 가격은 얼마인가요?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)